In [2]:
import numpy as np
import pandas as pd
import graphlab 
import sklearn 

In [3]:
sales=graphlab.SFrame('kc_house_data.gl/')
sales_pd=pd.read_csv('kc_house_data.csv')

This non-commercial license of GraphLab Create for academic use is assigned to kurianck.mail@gmail.com and will expire on March 07, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1522556475.log


In [4]:
train_data_gl,test_data_gl=sales.random_split(.8,seed=0)
## a function to get split test and train datasets for pandas dataframe
## its input is a pandas dataframe and out put are two dataframes
def get_random_split_pd(data,seed=0.8):
    random=np.random.rand(len(data))<seed
    return data[random],data[~random]
train_data_sk,test_data_sk=get_random_split_pd(sales_pd)

In [5]:
test_features=['sqft_living','bedrooms','bathrooms']

#from sklearn.linear_model import LinearRegression

#linreg=LinearRegression()
example_model_gl=graphlab.linear_regression.create(train_data_gl,target='price',features=test_features,validation_set=None)
#example_model_pd=linreg.fit()

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.059568     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [6]:
print example_model_gl.get("coefficients")

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



In [7]:
example_predictions_gl=example_model_gl.predict(test_data_gl)
print example_predictions_gl[0],test_data_gl[0]["price"]

350640.366016 310000.0


In [8]:
def get_residual_sum_of_squares(model,data,outcome):
    estimates=model.predict(data)
    return sum((np.array(estimates)-np.array(outcome))**2)

print get_residual_sum_of_squares(example_model_gl,test_data_gl,test_data_gl["price"])

273761538330190.6


In [16]:
model_1=['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']

train_data_gl['bed_bath_rooms']=train_data_gl['bedrooms']*train_data_gl['bathrooms']
model_2=model_1 +['bed_bath_rooms']

train_data_gl['bedrooms_squared']=train_data_gl['bedrooms'].apply(lambda x:x**2)

from math import log

train_data_gl['log_sqft_living']=train_data_gl['sqft_living'].apply(lambda x:log(x))

train_data_gl['lat_plus_long']=train_data_gl['lat']+train_data_gl['long']
model_3=model_2 +['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [30]:
model_1_gl=graphlab.linear_regression.create(train_data_gl,target='price',features=model_1,validation_set=None)
#print model_1_gl.get("coefficients")

model_2_gl=graphlab.linear_regression.create(train_data_gl,target='price',features=model_2,validation_set=None)
#print model_2_gl.get("coefficients")

model_3_gl=graphlab.linear_regression.create(train_data_gl,target='price',features=model_3,validation_set=None)
#print model_3_gl.get("coefficients")

test_data_gl['bed_bath_rooms']=test_data_gl['bedrooms']*test_data_gl['bathrooms']
test_data_gl['bedrooms_squared']=test_data_gl['bedrooms'].apply(lambda x:x**2)
test_data_gl['log_sqft_living']=test_data_gl['sqft_living'].apply(lambda x:log(x))
test_data_gl['lat_plus_long']=test_data_gl['lat']+test_data_gl['long']

print test_data_gl['bed_bath_rooms'].mean()                  
print test_data_gl['bedrooms_squared'].mean()                  
print test_data_gl['log_sqft_living'].mean()                  
print test_data_gl['lat_plus_long'].mean()                  


Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.057023     | 3193229.177908     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

7.50390163159
12.4466777016
7.55027467965
-74.6533349722


In [34]:
rss_test_model_1=get_residual_sum_of_squares(model_1_gl,test_data_gl,test_data_gl['price'])
rss_test_model_2=get_residual_sum_of_squares(model_2_gl,test_data_gl,test_data_gl['price'])
rss_test_model_3=get_residual_sum_of_squares(model_3_gl,test_data_gl,test_data_gl['price'])


print rss_test_model_1
print rss_test_model_2
print rss_test_model_3

226568089092808.25
224368799993634.34
251829318950283.66
